In [ ]:
!pip install pygame
!pip install pygad

!git clone https://github.com/mkaanaslan/evoman.git tmp
!cp -r /kaggle/working/tmp/* /kaggle/working/
!rm -R /kaggle/working/tmp

In [ ]:
import sys, os
import numpy as np

import random

from evoman.environment import Environment
from evoman.controller import Controller
from pygad.kerasga import model_weights_as_vector,model_weights_as_matrix
from deap import base, creator, tools, algorithms
import multiprocessing

In [ ]:
def sigmoid_activation(x):
	return 1./(1.+np.exp(-x))


class player_controller(Controller):
	def __init__(self, _n_hidden):
		self.n_hidden = [_n_hidden]

	def set(self,controller, n_inputs):
		# Number of hidden neurons
		
		if self.n_hidden[0] > 0:
			# Preparing the weights and biases from the controller of layer 1

			# Biases for the n hidden neurons
			self.bias1 = controller[:self.n_hidden[0]].reshape(1, self.n_hidden[0])
			# Weights for the connections from the inputs to the hidden nodes
			weights1_slice = n_inputs * self.n_hidden[0] + self.n_hidden[0]
			self.weights1 = controller[self.n_hidden[0]:weights1_slice].reshape((n_inputs, self.n_hidden[0]))

			# Outputs activation first layer.


			# Preparing the weights and biases from the controller of layer 2
			self.bias2 = controller[weights1_slice:weights1_slice + 5].reshape(1, 5)
			self.weights2 = controller[weights1_slice + 5:].reshape((self.n_hidden[0], 5))

	def control(self, inputs, controller):
		# Normalises the input using min-max scaling
		inputs = (inputs-min(inputs))/float((max(inputs)-min(inputs)))
		
		if self.n_hidden[0]>0:
			# Preparing the weights and biases from the controller of layer 1

			# Outputs activation first layer.
			output1 = sigmoid_activation(inputs.dot(self.weights1) + self.bias1)

			# Outputting activated second layer. Each entry in the output is an action
			output = sigmoid_activation(output1.dot(self.weights2)+ self.bias2)[0]
		else:
			bias = controller[:5].reshape(1, 5)
			weights = controller[5:].reshape((len(inputs), 5))

			output = sigmoid_activation(inputs.dot(weights) + bias)[0]

		# takes decisions about sprite actions
		if output[0] > 0.5:
			left = 1
		else:
			left = 0

		if output[1] > 0.5:
			right = 1
		else:
			right = 0

		if output[2] > 0.5:
			jump = 1
		else:
			jump = 0

		if output[3] > 0.5:
			shoot = 1
		else:
			shoot = 0

		if output[4] > 0.5:
			release = 1
		else:
			release = 0

		return [left, right, jump, shoot, release]

In [ ]:
def fitness_function(solution):
    return (env.play(np.array(solution))[0],)

In [ ]:
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"

env = Environment(playermode="ai",
				  player_controller=player_controller(0),
			  	  speed="fastest",
				  enemymode="static",
                  enemies="1",
				  level=2,
                  visuals=False)

In [ ]:
individual_size = 105

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("weight_bin", np.random.uniform,-1,1)   #Initiate weights from uniform distribution between -1 to +1
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.weight_bin, n=individual_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [ ]:
"""
def generate_individual(ind_class, size):
    initializer = tf.keras.initializers.HeUniform()
    individual = initializer(shape=(size,))
    return ind_class(individual.numpy().tolist())

toolbox.register('individual' ,generate_individual, creator.Individual, size=individual_size)
"""

In [ ]:
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitness_function)

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Min", np.min)

In [ ]:
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)

In [ ]:
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

In [ ]:
%%time
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.2, ngen=10, halloffame=hof, stats=stats,verbose=True)